## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import time
import os
from copy import copy

import astropy
from astropy.modeling import models
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.table import Table, vstack
from astropy.stats import sigma_clipped_stats
from astropy.nddata import StdDevUncertainty
from astropy.utils.data import download_file
from photutils import CircularAperture, SkyCircularAperture, DAOStarFinder
from regions import PixCoord, CirclePixelRegion, CircleSkyRegion
from astropy.io import fits, ascii
from astropy.wcs import WCS
from astropy.utils.data import get_pkg_data_filename
from astropy.utils.data import download_file

from specutils import Spectrum1D
from specutils.fitting import fit_generic_continuum, fit_lines
from specutils.manipulation import box_smooth, extract_region, SplineInterpolatedResampler
from specutils.analysis import line_flux, centroid, equivalent_width
from specutils.spectra import SpectralRegion
from specutils import SpectrumList

from scipy.optimize import curve_fit



from jdaviz import Imviz, Specviz, Cubeviz
from jdaviz.app import Application
imviz = Imviz()
specviz = Specviz()
cubeviz = Cubeviz()



C:\Users\Dan\AppData\Local\Temp\ipykernel_116\2330992003.py:16: DeprecationWarning: `photutils.CircularAperture` is a deprecated alias for `photutils.aperture.CircularAperture` and will be removed in the future. Instead, please use `from photutils.aperture import CircularAperture` to silence this warning.
  from photutils import CircularAperture, SkyCircularAperture, DAOStarFinder
C:\Users\Dan\AppData\Local\Temp\ipykernel_116\2330992003.py:16: DeprecationWarning: `photutils.SkyCircularAperture` is a deprecated alias for `photutils.aperture.SkyCircularAperture` and will be removed in the future. Instead, please use `from photutils.aperture import SkyCircularAperture` to silence this warning.
  from photutils import CircularAperture, SkyCircularAperture, DAOStarFinder
C:\Users\Dan\AppData\Local\Temp\ipykernel_116\2330992003.py:16: DeprecationWarning: `photutils.DAOStarFinder` is a deprecated alias for `photutils.detection.DAOStarFinder` and will be removed in the future. Instead, please 

## Imviz


#### To import JWST images
Find images as .fits files on https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html

Upload to notebook by running cell below

In [3]:
imviz.show_in_sidecar()



Application(config='imviz', events=['call_viewer_method', 'close_snackbar_message', 'data_item_remove', 'data_…

In [4]:
for file in files:
    imviz.load_data(file)
    print(file,' loaded')
print('All files loaded!')

viewer = imviz.default_viewer
viewer.cuts = '99%'

C:\Users\Dan\miniconda3\envs\jdaviz-env\lib\site-packages\glue\viewers\common\viewer.py:186: UserWarning: Add large data set?
  warnings.warn(message)


./Fits/25/1.fits  loaded


C:\Users\Dan\miniconda3\envs\jdaviz-env\lib\site-packages\glue\viewers\common\viewer.py:186: UserWarning: Add large data set?
  warnings.warn(message)


./Fits/25/2.fits  loaded


C:\Users\Dan\miniconda3\envs\jdaviz-env\lib\site-packages\glue\viewers\common\viewer.py:186: UserWarning: Add large data set?
  warnings.warn(message)


./Fits/25/3.fits  loaded
./Fits/25/4.fits  loaded
./Fits/25/5.fits  loaded
./Fits/25/6.fits  loaded
./Fits/25/7.fits  loaded
./Fits/26/1.fits  loaded
./Fits/26/2.fits  loaded
./Fits/26/3.fits  loaded
All files loaded!
